In [85]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 

In [86]:
def convert_mat_np(gender=None):
    f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
    m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat"
    # m_file = "../raw_data/Emotrans1_Boy_data_preprocessed_33.mat"
    # f_file = "../raw_data/Emotrans1_girl_data_preprocessed_33.mat"
    if gender == "f":
        data_dict_female = mat73.loadmat(f_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])
    elif gender == 'm':
        data_dict_female = mat73.loadmat(m_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])  
    else: 
        raise Exception("gender not defined")

In [87]:
m = convert_mat_np("m")
f = convert_mat_np("f")

In [9]:
# # boy 18, 19, 21
# # delete and put to the end of firl 
# new_f = [] 
# for i in range(f.shape[0]):
#     new_f.append(f[i])
# for i in [17,18,20]:
#     new_f.append(m[i])
# f = np.array(new_f)
# # new_arr = []
# m = np.delete(m,[17,18,20],axis=0)
# print(f.shape)
# print(m.shape)
# # deleted.shape

(22, 4, 4)
(20, 4, 4)


In [88]:
# type: 0 --> average 3+4 and 1+2; 1--> use only 3 and 2  
def get_pos_or_neg(data,position,type=0):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        if type == 0:
            diff = (data[i][position[0][0]][position[0][1]]+data[i][position[1][0]][position[1][1]])/2 - (data[i][position[3][0]][position[3][1]]+data[i][position[2][0]][position[2][1]])/2
        elif type == 1:
             diff = (data[i][position[1][0]][position[1][1]] - data[i][position[2][0]][position[2][1]])/2
        arr.append(diff)
    return np.array(arr)

In [89]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
# output: x 
def choose_freq(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 1, axis=3)
        return data
    elif sec == 1:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 0, axis=3) 
        return data
    else:
        print('sec not specified')
        return data 

In [90]:
# reshape to 128*8*2 = 2048
# return x 
def squeeze_feature_size(data):
    new_data = []
    for d in range(data.shape[0]):
#         for i in range(df[d].shape[0]):
        size = data[d].shape
        new_data.append(data[d].reshape(size[0], size[1]*size[2]*size[3]))  
    return np.array(new_data)

In [91]:
# generate labels for x 
# input: data and label_type (0:zeros, 1: ones )
def generate_labels(df,type=None):
    arr = []
    label = []
    label_type = -1
    if type == 0:
        label_type = 0
    elif type == 1:
        label_type = 1
    else:
        print("Wrong type - get_pos_or_neg()")
        return 
    for d in range(df.shape[0]):
#         for i in range(df[d].shape[0]):
        size = df[d].shape
#         print(size)
        if label_type == 1:
            label.append(np.ones((size[0],1)))
        else:
            label.append(np.zeros((size[0],1)))
    return np.array(label)

In [92]:
# combine neg and postive for boys and girls separately
def combine_net_neg(netX,netY,negX,negY):
    new_x, new_y = [],[]
    for i in range(netX.shape[0]):
        new_x.append(np.concatenate((netX[i],negX[i]),axis=0))
        new_y.append(np.concatenate((netY[i],negY[i]),axis=0))
    return np.array(new_x),np.array(new_y)

In [93]:
# get x and y training set for leave one out 
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    return new_x,new_y

In [94]:
def calculate(s):
    num, op, stack = 0, '+', [0]
    ops = {'+':lambda x, y: y, '-':lambda x, y: -y, '*':lambda x, y: x*y, '/':lambda x, y: (int)(float(x)/float(y))}
    for i, c in enumerate(s):
        if c.isdigit():
            num = num * 10 + int(c)
        if not c.isdigit() and c != ' ' or i == len(s)-1:
            prev = 0 if op in '+-' else stack.pop()
            stack.append(ops[op](prev, num))
            num, op = 0, c
    return sum(stack)
def split_feature(str):
    arr = str.split(',')
    new_arr=[]
    for i in arr:
        if "-" in i:
            i = i.split('-')
            if "*" in i[0] or "+" in i[0]:
                i[0] = calculate(i[0])
            if "*" in i[1] or "+" in i[1]:
                i[1] = calculate(i[1])
            new_arr.append([int(i[0])-1,int(i[1])-1])
    return new_arr

In [95]:
parietal = "32-5,31-18,30-17,29-16,28-15,27-14,26-13,32+3-6,32+4-7,32+5-8,32+6-9,32+7-10,32+8-11,32+9-12,32+13-32*3+29,32+12-32*3+30,32+11-32*3+31,32+10-32*3+32,32+18-32*3+28, 32+17 - 32*3+27, 32+16 - 32*3 + 26, 32+15 - 32*3+25,32+14-32*3+24"
frontal = "32*2 + 16 - 32*2 + 29, 32*2 + 15 - 32*2 + 28, 32*2 + 14 - 32*2 + 27, 32*2 + 13 - 32*2 + 26, 32*2 + 12 - 32*2 + 25, 32*2 + 11 - 32*2 + 24,32*2  + 8 - 32*2 +30,32*2  + 9 - 32*2 +31,32*2  + 10 - 32*2 +32,32*2  + 4 - 32*3 +4,32*2  + 3 - 32*3 +3,32*2  + 7 - 32*3 +7,32*2  + 6 - 32*3 +6,32*2  + 5 - 32*3 +5,32*2  + 7 - 32*3 +7,32  +27 - 32*3 +8,32  +28 - 32*3 +9,32  +29 - 32*3 +10,32  + 30 - 32*3 +11,32  + 31 - 32*3 +12,32  + 33 - 32*3 +13"
middle = "32  +26 - 32*3 +23,32  +25 - 32*3 +22,32  +24 - 32*3 +21,32  + 23 - 32*3 +20,32  + 22 - 32*3 +19,32  + 21 - 32*3 +18"
parietal = split_feature(parietal)
frontal = split_feature(frontal)
middle = split_feature(middle)
print(parietal)
print(frontal)
print(middle)
chosen_feature = parietal+frontal+middle

[[31, 4], [30, 17], [29, 16], [28, 15], [27, 14], [26, 13], [25, 12], [34, 5], [35, 6], [36, 7], [37, 8], [38, 9], [39, 10], [40, 11], [44, 124], [43, 125], [42, 126], [41, 127], [49, 123], [48, 122], [47, 121], [46, 120], [45, 119]]
[[79, 92], [78, 91], [77, 90], [76, 89], [75, 88], [74, 87], [71, 93], [72, 94], [73, 95], [67, 99], [66, 98], [70, 102], [69, 101], [68, 100], [70, 102], [58, 103], [59, 104], [60, 105], [61, 106], [62, 107], [64, 108]]
[[57, 118], [56, 117], [55, 116], [54, 115], [53, 114], [52, 113]]


In [96]:
# define model 
model = RandomForestClassifier(n_estimators=150) 

In [97]:
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
pos_df_m = get_pos_or_neg(m,pos,1)
neg_df_m = get_pos_or_neg(m,neg,1)
pos_df_f = get_pos_or_neg(f,pos,1)
neg_df_f = get_pos_or_neg(f,neg,1)


In [98]:
pos_df_m[0].shape

(18, 128, 8, 2)

In [99]:
temp = pos_df_m[0][0][31]-pos_df_m[0][0][4]
temp.shape
"""
new_arr=[]
itearte through chosen_feature
    append to new_arr
return new_arr 
"""
def cal_diff(features,arr):
    new_arr = [] 
    for i in features:
        new_arr.append(arr[i[0]]-arr[i[1]])
    return np.array(new_arr)

def get_new_features(df,features):
    new_df=[]
    for i in range(df.shape[0]):
        temp_df = []
        for j in range(df[i].shape[0]):
            temp_df.append(cal_diff(features,df[i][j]))
        new_df.append(np.array(temp_df))
    return np.array(new_df)
temp = get_new_features(pos_df_m,chosen_feature)
temp.shape

(23,)

In [100]:
def model_train(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train.ravel())
    y_pred = model.predict(x_test)
    return y_test, y_pred

In [101]:
# get x train and test set for witin CV 
def get_x_train_test(data,index):
    x_test = data[index]
    x_train = np.delete(data,index,axis=0)
    return x_train,x_test.reshape(1,-1)

# get y train and test set for witin CV 
def get_y_train_test(data,index):
    y_test = data[index]
    y_train = np.delete(data,index,axis=0)
    return y_train.astype(int).ravel(),y_test.astype(int)

In [102]:
pos_df_m = get_new_features(pos_df_m,chosen_feature)
neg_df_m = get_new_features(neg_df_m,chosen_feature)
pos_df_f = get_new_features(pos_df_f,chosen_feature)
neg_df_f = get_new_features(neg_df_f,chosen_feature)

In [103]:
pos_sequeezed_m = squeeze_feature_size(pos_df_m)
neg_sequeezed_m = squeeze_feature_size(neg_df_m)
pos_sequeezed_f = squeeze_feature_size(pos_df_f)
neg_sequeezed_f = squeeze_feature_size(neg_df_f)

In [104]:
pos_labels_m = generate_labels(pos_sequeezed_m,1)
neg_labels_m = generate_labels(neg_sequeezed_m,0)
pos_labels_f = generate_labels(pos_sequeezed_f,1)
neg_labels_f = generate_labels(neg_sequeezed_f,0)

In [105]:
boy_x,boy_y = combine_net_neg(pos_sequeezed_m,pos_labels_m,neg_sequeezed_m,neg_labels_m)
girl_x,girl_y = combine_net_neg(pos_sequeezed_f,pos_labels_f,neg_sequeezed_f,neg_labels_f)

In [106]:
# combine boys and girls 
x = np.concatenate((boy_x,girl_x),axis=0)
y = np.concatenate((boy_y,girl_y),axis=0)

In [107]:
all_accuracy=[]
for d in range(x.shape[0]):
    print("leave one out # person:",d)
    if d == 0:
        x_train,y_train = get_x_y_train(x[d+1:],y[d+1:])
        x_test,y_test =x[d],y[d]
    elif d == x.shape[0]-1:
        x_train,y_train = get_x_y_train(x[:d],y[:d])
        x_test,y_test = x[d],y[d]
    else:
        x_train,y_train = get_x_y_train(np.append(x[:d],x[d+1:],axis=0),np.append(y[:d],y[d+1:],axis=0))
        x_test,y_test = x[d],y[d]
    # print(x_train.shape,x_test.shape)
    # print(y_train.shape,y_test.shape)
    # return
    y_test,y_pred=model_train(x_train,x_test,y_train,y_test,model)
    accuracy = accuracy_score(y_test,y_pred)
    all_accuracy.append(accuracy)
print(all_accuracy)

leave one out # person: 0
leave one out # person: 1
leave one out # person: 2
leave one out # person: 3
leave one out # person: 4
leave one out # person: 5
leave one out # person: 6
leave one out # person: 7
leave one out # person: 8
leave one out # person: 9
leave one out # person: 10
leave one out # person: 11
leave one out # person: 12
leave one out # person: 13
leave one out # person: 14
leave one out # person: 15
leave one out # person: 16
leave one out # person: 17
leave one out # person: 18
leave one out # person: 19
leave one out # person: 20
leave one out # person: 21
leave one out # person: 22
leave one out # person: 23
leave one out # person: 24
leave one out # person: 25
leave one out # person: 26
leave one out # person: 27
leave one out # person: 28
leave one out # person: 29
leave one out # person: 30
leave one out # person: 31
leave one out # person: 32
leave one out # person: 33
leave one out # person: 34
leave one out # person: 35
leave one out # person: 36
leave one o

In [118]:
"""
Within 
"""
all_accuracy = []
# train a model 
# return y_test, y_pred 
avg_accuracy = []
for d in range(x.shape[0]):
    # training set and test set
    accuracy =[]
    print("within # person:",d)
    for i in range(x[d].shape[0]):
        x_train,x_test = get_x_train_test(x[d],i)
        y_train,y_test = get_y_train_test(y[d],i)
        y_test,y_pred = model_train(x_train,x_test,y_train,y_test,model)
        accuracy.append(accuracy_score(y_test,y_pred))
    avg_accuracy.append(sum(accuracy)/len(accuracy))
    all_accuracy.append(accuracy)

within # person: 0
within # person: 1
within # person: 2
within # person: 3
within # person: 4
within # person: 5
within # person: 6
within # person: 7
within # person: 8
within # person: 9
within # person: 10
within # person: 11
within # person: 12
within # person: 13
within # person: 14
within # person: 15
within # person: 16
within # person: 17
within # person: 18
within # person: 19
within # person: 20
within # person: 21
within # person: 22
within # person: 23
within # person: 24
within # person: 25
within # person: 26
within # person: 27
within # person: 28
within # person: 29
within # person: 30
within # person: 31
within # person: 32
within # person: 33
within # person: 34
within # person: 35
within # person: 36
within # person: 37
within # person: 38
within # person: 39
within # person: 40
within # person: 41


In [119]:
for i in range(len(avg_accuracy)):
    avg_accuracy[i] = round(avg_accuracy[i],3)

In [120]:
df = pd.read_csv('feb_27.csv')

In [121]:
print(avg_accuracy)
df.columns

[0.611, 0.583, 0.5, 0.25, 0.611, 0.667, 0.583, 0.556, 0.583, 0.5, 0.533, 0.667, 0.611, 0.433, 0.639, 0.417, 0.639, 0.444, 0.292, 0.778, 0.556, 0.528, 0.417, 0.667, 0.611, 0.611, 0.528, 0.333, 0.861, 0.556, 0.528, 0.667, 0.5, 0.361, 0.639, 0.611, 0.333, 0.417, 0.583, 0.6, 0.633, 0.467]


Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Exp_ID', 'exp_time',
       'trais no.', 'note', 'signal_condition', 'gender', 'age', 'race',
       'handiness', 'sleep condition', 'Q11', 'Q12', 'Q13', 'Q14', 'ID',
       'ID_boy', 'ID_girl', 'within_classification', 'within_Feature_asymmtry',
       'stress', 'stressGroup', '42set_ID', 'between_classification_all',
       'between_Feature_asymmtry', 'between_stressGroup', 'between_gender',
       '39set_ID', 'between_classification', 'between_stressGroup.1',
       '33set_ID', 'between_classification.1', 'between_stressGroup.2',
       'Unnamed: 32', 'Unnamed: 33'],
      dtype='object')

In [92]:
for _ in range(22):
    avg_accuracy.append(-1)

In [122]:
for i in range(4):
    avg_accuracy.append(-1)
df["within_Feature_asymmtry"] = avg_accuracy[:46]

In [123]:
df.to_csv('feb_27.csv')

In [97]:
sum(accs[:42])/len(accs[:42])

0.6708571428571426

In [96]:
accs

[0.722,
 0.611,
 0.722,
 0.611,
 0.722,
 0.722,
 0.861,
 0.639,
 0.667,
 0.639,
 0.8,
 0.5,
 0.583,
 0.667,
 0.611,
 0.639,
 0.694,
 0.5,
 0.583,
 0.583,
 0.722,
 0.806,
 0.639,
 0.694,
 0.667,
 0.611,
 0.528,
 0.778,
 0.778,
 0.639,
 0.778,
 0.694,
 0.722,
 0.5,
 0.778,
 0.833,
 0.7,
 0.7,
 0.533,
 0.639,
 0.667,
 0.694,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]